#### Método de elementos finitos. Código para la solución de la ecuación diferencial en 1D:
$$-\frac{d}{dx}\left[a(x)\frac{du}{dx}\right]+c(x)u=q(x)$$


Andrea Jara 
<br>
Proyecto 1
<br>
ICYA4414 - Modelación con elementos finitos
<br>
Universidad de los Andes
<br>
Marzo, 2020

In [1]:
%pylab inline
import numpy as np
import matplotlib

Populating the interactive namespace from numpy and matplotlib


Al correr la celda de abajo se generará un archivo de texto (input.txt), en la carpeta donde se encuentra guardado este script, al que debe modificarse la segunda fila con la longitud del dominio, el número de elementos en el que se quiere dividir el dominio, el orden de aproximación (1 para lineal y 2 para cuadrático) y el número de puntos para la integración que se realiza con la cuadratura de Gauss. 

In [2]:
#Archivo de entrada 
filename = "input.txt"
outfile = open(filename,'w')

outfile.write("Longitud_del_dominio ")
outfile.write("Numero_de_elementos ")
outfile.write("Orden_de_aproximacion ")
outfile.write("Puntos_para_integracion\n")

for i in range(0,4):
    outfile.write("0 ")

outfile.close()

In [3]:
#Archivo de salida 1
filename = "ODE21D_1.txt"
outfile = open(filename,'w')

f = open("input.txt","r")
out = f.readlines()
values = out[1].split()

num_elem = int(values[1])
ord_aprox = int(values[2])

num_nodos = num_elem*(ord_aprox + 1) - (num_elem - 1)

outfile.write("Numero_de_elementos ")
outfile.write("Numero_de_nodos ")
outfile.write("Orden_de_aproximacion ")
outfile.write("Puntos_para_integracion\n")

outfile.write(values[1] + " ")
outfile.write(str(num_nodos) + " ")
outfile.write(values[2] + " ")
outfile.write(values[3] + " ")

outfile.close()

El archivo que se genera al correr la celda de abajo (ODE21D_coord.txt) debe ser modificado con las condiciones de borde esenciales y naturales en los nudos donde se conozca la variable principal $u$ o la variable secundaria $a(x)\frac{du}{dx}$. 
<br>
<br>
Para esto, se modifica la columna Condicion_de_borde en la fila de la coordenada donde se conozca alguna de las variables, y se modifica la columna CB_esencial o la columna CB_natural con 1 dependiendo de si el valor corresponde a una condición de borde esencial o a una condición de borde natural.
<br>
<br>
Además, si la condición de borde depende de la solución en alguno de los nudos:

$$\left[\frac{dT}{dx}+\frac{\beta}{k}\left(T-T_{a}\right)\right]=0$$
<br>
se debe modificar la columna Coeficiente_de_rigidez_convectivo con el valor de $\frac{\beta}{k}$.

In [4]:
#Archivo de salida 2

filename = "ODE21D_coord.txt"
outfile = open(filename,'w')

f = open("input.txt","r")
out = f.readlines()
values = out[1].split()

outfile.write("Nudo ")
outfile.write("Coordenada_x ")
outfile.write("Condicion_de_borde ")
outfile.write("CB_esencial ")
outfile.write("CB_natural ")
outfile.write("Coeficiente_de_rigidez_convectivo\n")

long_dominio = float(values[0])
num_elementos = int(values[1])
ord_aprox = int(values[2])

num_nodos = num_elem*(ord_aprox + 1) - (num_elem - 1)
long_elem = long_dominio/num_elementos

for i in range(1, num_nodos + 1):
    outfile.write(str(i) + " ")
    
    k = i - 1
    outfile.write(str(k*(long_elem/ord_aprox)) + " ")
    outfile.write("0 0 0 0\n")

outfile.close()

El archivo que se generará al correr la celda de abajo (ODE21D_conect.txt) contiene el número del elemento con los números de los nodos que lo conforman.

In [5]:
#Archivo de salida 3

filename = "ODE21D_conect.txt"
outfile = open(filename,'w')

f = open("input.txt","r")
out = f.readlines()
values = out[1].split()

num_elementos = int(values[1])
ord_aprox = int(values[2])

outfile.write("Elemento ")
outfile.write("Nudos\n")

for i in range(1,num_elementos+1):
    outfile.write(str(i) + " ")
    
    if ord_aprox == 1:
        outfile.write(str(i) + "," + str(i + 1) + "\n")
    else:
        k=i-1
        outfile.write(str(2*k + 1) + "," + str(2*k + 2) + "," + str(2*k + 3)+"\n")

outfile.close()

#### Modificar la celda de abajo por la función $a(x)$

In [6]:
def a(x):
    a = 1
    return a

#### Modificar la celda de abajo por la función $c(x)$

In [7]:
def c(x):
    c = 400
    return c

#### Modificar la celda de abajo por la función $q(x)$

In [8]:
def q(x):
    q = 0
    return q

In [9]:
def Gauss(npoints):
    return np.polynomial.legendre.leggauss(npoints)

In [10]:
#PG: array con puntos de Gauss
#np: numero de puntos de Gauss
#ord_aprox: orden de aproximacion (1 para lineal, 2 para cuadratico)

def interp(PG,npoints,ord_aprox):
    if ord_aprox==1:
        psi_1 = [(1 - gp)/2 for gp in PG]
        psi_2 = [(1 + gp)/2 for gp in PG]
        
        dpsi_1 = [-1/2 for i in range(0,npoints)]
        dpsi_2 = [1/2 for i in range(0,npoints)]
        
        psi = [psi_1,psi_2]
        dpsi = [dpsi_1,dpsi_2]
        
    elif ord_aprox == 2:
        psi_1 = [-gp*(1 - gp)/2 for gp in PG]
        psi_2 = [(1 - gp**2) for gp in PG]
        psi_3 = [gp*(1 + gp)/2 for gp in PG]
        
        dpsi_1 = [gp - 1/2 for gp in PG]
        dpsi_2 = [-2*gp for gp in PG]
        dpsi_3 = [gp + 1/2 for gp in PG]
        
        psi = [psi_1,psi_2,psi_3]
        dpsi = [dpsi_1,dpsi_2,dpsi_3]
        
    
    return [psi,dpsi]

La celda de abajo contiene el código para la solución de la ecuación diferencial de arriba. Al correrla se generará un archivo de texto (ODE21D_resultados.txt) que contiene la solución en los nodos de la variable principal, la derivada de la variable principal (por izquierda y por derecha porque los elementos consecutivos comparten un nodo), el valor de la variable secundaria calculada con la definición y el valor de la variable secundaria calculada por equilibrio.
<br>
<br>
El parámetro de entrada Ta corresponde a $T_{a}$ para los casos en donde se tenga una condición de borde como
$$\left[\frac{dT}{dx}+\frac{\beta}{k}\left(T-T_{a}\right)\right]=0$$

In [11]:
def ED2O_1D(Ta):
    
    file = open("ODE21D_coord.txt","r")
    out = file.readlines()
    out = out[1:]
       
    xa = [float(x.split(' ')[1]) for x in out]
    
    condicion_de_borde = [float(cb.split(' ')[2]) for cb in out]
    CB_esencial = [float(cb.split(' ')[3]) for cb in out]
    CB_natural = [float(cb.split(' ')[4]) for cb in out]
    CRC = [float(crc.split(' ')[5]) for crc in out]
    
    file.close()
    
    file = open("ODE21D_1.txt")
    out = file.readlines()
    
    values = out[1].split()
    
    num_elem = int(values[0])
    num_nodos = int(values[1])
    ord_aprox = int(values[2])
    npoints = int(values[3])
    
    file.close()
    
    GP,w = Gauss(npoints)
    psi,dpsi = interp(GP, npoints, ord_aprox)
    
    he = xa[ord_aprox] - xa[0]
    J = he/2
    
#_________________________________________________________Matriz K__________________________________________________________    
    K = zeros([num_nodos, num_nodos])
    
    m = [[ord_aprox*i + k for k in range(ord_aprox + 1)] for i in range(0, num_elem)]
    
    for l in range(0, num_elem):
        
        Ke=zeros([ord_aprox + 1,ord_aprox + 1])
        
        for i in range(0,ord_aprox + 1):
            for j in range(0,ord_aprox + 1):
                dpsi_i = dpsi[i]
                dpsi_j = dpsi[j]
            
                psi_i = psi[i]
                psi_j = psi[j]
            
                Ke[i,j]=sum([(a(xa[l*ord_aprox] + he*(1 + GP[k])/2)*(1/J)*dpsi_i[k]*(1/J)*dpsi_j[k] + 
                              c(xa[l*ord_aprox] + he*(1 + GP[k])/2)*psi_i[k]*psi_j[k])*J*w[k] for k in range(0, size(GP))])
                
                K[m[l][i],m[l][j]] = K[m[l][i], m[l][j]] + Ke[i,j]
                
    print("K")
    print(K)

#_________________________________________________________Vector F__________________________________________________________
    F = zeros([num_nodos,1])
    
    for l in range(0, num_elem):
        
        Fe = zeros([ord_aprox + 1])
        
        for i in range(0, ord_aprox + 1):
            psi_i = psi[i]
            
            Fe[i] = sum([q(xa[l*ord_aprox] + he*(1 + GP[k])/2)*psi_i[k]*J*w[k] for k in range(0,size(GP))])
            
            F[m[l][i],0] = F[m[l][i],0] + Fe[i]
            
    print("F")        
    print(F)
    
#_______________________________________________Condicion de borde natural__________________________________________________
    Q = zeros([num_nodos,1])   
    
    for i in range(0,num_nodos):
        if CB_natural[i] == 1:
            Q[i,0] = condicion_de_borde[i]
        else:
            Q[i,0] = 0
            
    print("Q")
    print(Q)
#________________________________________________Condicion de borde esencial________________________________________________            
    F_Q = F + Q
    K_n = K.copy()
    
    index = []
    
    for i in range(0,num_nodos):
        if CB_esencial[i] == 1:
            K_n[:,i] = 0
            K_n[i,:] = 0
            K_n[i,i] = 1
            F_Q[i] = condicion_de_borde[i]
            index += [i]
            
    for i in range(0,num_nodos):
        if CB_esencial[i] == 0:
            F_Q[i] = F_Q[i] - sum([K[i,p]*condicion_de_borde[p] for p in index])
    
    print("K_n")
    print(K_n)
    
    print("F+Q")
    print(F_Q)
    
#______________________________________________Coeficiente de rigidez convectivo____________________________________________            
    for i in range(0,num_nodos):
        if CRC[i] != 0:
            K_n[i,i] = K_n[i,i] + CRC[i]
            F_Q[i] = F_Q[i] + CRC[i]*Ta
            
#_____________________________________________________Vector U______________________________________________________________    
    U = dot(inv(K_n),F_Q)
    
    print("U")
    print(U)

#____________________________________________________Vector dU______________________________________________________________
    dU = zeros([num_elem*(ord_aprox + 1), 1])
    
    ksi = [-1, 0, 1]
    
    xa_ksi = []
    
    for l in range(0, num_elem):
        for i in range(0, ord_aprox + 1):
            xa_ksi.append(ksi[i])

    psi_xa, dpsi_xa = interp(xa_ksi, size(xa_ksi), ord_aprox)
    
    j = 0
    for l in range(0, num_elem):
        for i in range(0, ord_aprox + 1):
            dU[j, 0] = sum([U[l*ord_aprox + k][0]*dpsi_xa[k][j]*1/J for k in range(ord_aprox + 1)])
            j = j + 1
            
    
    print("dU")
    print(dU)
    
#____________________________________________________Vector Q_______________________________________________________________    
    Q[0] = (-1)*a(xa[0])*dU[0]
    Q[-1] = a(xa[-1])*dU[-1]
    
    print("Q")
    print(Q)

#____________________________________________________Vector FE______________________________________________________________
    FE_1 = dot(K,U) - F
    
    FE = Q.copy()
    FE[0] = FE_1[0]
    FE[-1] = FE_1[-1]
    
    print("Fe")
    print(FE)
    
    filename = "ODE21D_resultados.txt"
    outfile = open(filename,'w')

    outfile.write("Nudo ")
    outfile.write("Coordenada ")
    outfile.write("Valor_de_VD ")
    outfile.write("Valor_de_dVD ")
    outfile.write("Valor_de_Q ")
    outfile.write("Valor_de_FE\n")
    
    k = 0
    for i in range(0,num_nodos):
        outfile.write(str(i + 1) + " ")
        outfile.write(str(xa[i]) + " ")
        outfile.write(str(U[i][0]) + " ")
        
        if i == 0:
            outfile.write(str(dU[0])+" ")
        elif i == num_nodos - 1:
            outfile.write(str(dU[-1])+" ")
        else:
            if (i)%(ord_aprox) == 0:
                outfile.write(str(dU[k]) + "," + str(dU[k + 1]) + " ") 
                k = k + 1
            else:
                outfile.write(str(dU[k])+" ")
        k = k + 1
        
        outfile.write(str(Q[i][0]) + "\n")
        outfile.write(str(FE[i][0]) + "\n")
        
    
    outfile.close()
        

#### Ejemplo (Problema 4.7 Reddy - An introduction to the finite element method, 4 ed.) 
<br>
Una barra de acero con diámetro $d=2 cm$, longitud $L=5 cm$, y conductividad térmica $k=50 W/m°C$ está expuesta a una temperatura ambiente $T_{\inf}=20 °C$ con un coeficiente de transferencia de calor $\beta=100 W/m^2°C$. El extremo izquierdo de la barra es mantenido a una temperatura de 320 °C. Determine la temperatura a 2.5 cm y 5.0 cm a partir el extremo izquierdo de la barra y el flujo de calor en el extremo izquierdo. 
<br>
<br>
La ecuación que gobierna este problema está dada por:

$$-\frac{d^2\theta}{dx^2}+m^2\theta=0$$
<br>
donde
$$\theta(0)=T(0)-T_{\inf}=300 °C$$
y
$$\left[\frac{d\theta}{dx}+\frac{\beta}{k}\theta\right]=0$$ en $x=L$

Se tiene que $a(x)=1$, $c(x)=m^2=400 m^{-2}$ y $q(x)=0$ y los archivos de texto que vienen dentro de la carpeta junto con este notebook. En la celda de abajo se presenta la solución para 1 elemento cuadrático.

In [12]:
ED2O_1D(0)

K
[[  94.66666667 -106.           13.            0.            0.        ]
 [-106.          218.66666667 -106.            0.            0.        ]
 [  13.         -106.          189.33333333 -106.           13.        ]
 [   0.            0.         -106.          218.66666667 -106.        ]
 [   0.            0.           13.         -106.           94.66666667]]
F
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Q
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
K_n
[[   1.            0.            0.            0.            0.        ]
 [   0.          218.66666667 -106.            0.            0.        ]
 [   0.         -106.          189.33333333 -106.           13.        ]
 [   0.            0.         -106.          218.66666667 -106.        ]
 [   0.            0.           13.         -106.           94.66666667]]
F+Q
[[  300.]
 [31800.]
 [-3900.]
 [    0.]
 [    0.]]
U
[[300.        ]
 [248.74449821]
 [213.13330448]
 [190.8958447 ]
 [180.6644129 ]]
dU
[[-4726.21246601]
 [-3474.66782095]
 [-2223.12317588]
 